In [365]:
import os
import numpy as np
np.random.seed(43)

In [366]:
setup_string = '''
# General setup
open morph-open-closed/open-closed.pdb
center
turn y 60

# Open bound structure get position of the ligand, missing in the morph
open 1ake
delete #1:.B
addh spec #1&protein hbond false
mmaker #0.1 #1

# Window appearance
windowsize 800 600
lighting reflectivity 0
set bg_color white
set silhouette
set silhouette_width 1.5

# Model appearance
color grey

# Add caption
2dlabel create title text "Binding and release of substrate leads to directional motion" xpos 0.1 ypos 0.92 color black

# Play movie sequence
2dlabel change title visibility show

# Hide everything
~ribbon; ~display
# Except ligand
display #1:AP5; color red #1:AP5; color byhet #1:AP5
'''

In [367]:
def pause_and_fade(pause_length, total_frames, file):
    file.write('# Fade out...\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format((pause + 1)* transparency_increment)
                  )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    file.write('~display #1:AP5\n')
    return total_frames

In [368]:
def pause_and_fade_in(pause_length, total_frames, file):
    file.write('# Fade in...\n')
    file.write('display #1:AP5\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format(100 - ((pause + 1)* transparency_increment))
        )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )

    return total_frames


def pause(pause_length, total_frames, file):
    for pause in np.arange(pause_length):
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    return total_frames

In [369]:
def add_bump(positive):
    sigma = 2
    if positive:
        mu = 6.0
    else:
        mu = -6.0
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

In [472]:
def rotate_forward(models, residues, positives, file, total_frames):
    #
    positions = np.empty(( len(residues), len(models) + 1))
    positions[:, 0] = 0
    #
    for (model_index, model) in enumerate(models, 1):
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues): 
            # Now keep incrementing this!
            positions[residue_index, model_index] = \
                            positions[residue_index, model_index - 1] + \
                            add_bump(positives[residue_index])

            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            colors = ['blue', 'green', 'purple', 'orange', 'yellow']
            file.write('rotation 2 {} 1; wait 1\n'.format(positions[residue_index, model_index]))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')

            #
            print('Frame {} → {}'.format(model + 1, positions[residue_index, model_index]))
            #
            
        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )

    return positions, total_frames

In [473]:
def rotate_backward(models, residues, positives, file, forward_rotations, total_frames):
    #
    positions = np.empty(( len(residues), len(models) + 1))
    positions[:, 0] = forward_rotations[:, -1]
    #
    # Now keep incrementing this!
    #
    for (model_index, model) in enumerate(models[::-1], 1):
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues):
            if model_index == 1:
                print("This angle's last position is... {}".format(positions[residue_index, 0]))
            
            positions[residue_index, model_index] = \
                            positions[residue_index, model_index - 1] + \
                            add_bump(positives[residue_index])

            #
            additional_rotation = positions[residue_index, model_index] - \
                                  forward_rotations[residue_index, -model_index]
            print('Frame {} → {}, which is a rotation of additional {}'.format(model + 1, 
                                                                               positions[residue_index, model_index],
                                                                               additional_rotation))            
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            colors = ['blue', 'green', 'purple', 'orange', 'yellow']
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
            )
    
    return positions, total_frames

In [475]:
residues = [20, 50, 120, 141, 200]
positives = [True, False, False, True, True]
pause_length = 20
morphs = np.arange(30)
total_frames = 0


with open('morph_setup.cmd', 'w') as file:
    file.write(setup_string)

    forward_rotations, total_frames = rotate_forward(
        morphs,
        residues,
        positives, 
        file, 
        total_frames)
    
    
    
    print(forward_rotations)
    
    total_frames = pause_and_fade(5, total_frames, file)
    backward_rotations, total_frames = rotate_backward(
        morphs, 
        residues, 
        positives, 
        file, 
        forward_rotations, 
        total_frames)

Frame 1 → 8.710524175769567
Frame 1 → -10.406125772985419
Frame 1 → -5.3904595867978875
Frame 1 → 6.2307786930237485
Frame 1 → 9.494871756912618
Frame 2 → 11.52926658334192
Frame 2 → -18.037891355130505
Frame 2 → -8.643660345606124
Frame 2 → 12.490598259329488
Frame 2 → 15.041847271652975
Frame 3 → 16.786506205537215
Frame 3 → -22.989193944893433
Frame 3 → -13.794416308683566
Frame 3 → 17.591220763412835
Frame 3 → 20.631896937232874
Frame 4 → 20.890471181667365
Frame 4 → -26.66278922858921
Frame 4 → -21.99273097144444
Frame 4 → 23.652921227603642
Frame 4 → 28.141075769669108
Frame 5 → 28.915387702236938
Frame 5 → -35.10483650845513
Frame 5 → -27.650414895452123
Frame 5 → 30.209315586356936
Frame 5 → 35.497445184164796
Frame 6 → 33.204277919883125
Frame 6 → -41.01302375621959
Frame 6 → -33.8668488262554
Frame 6 → 34.88044644615533
Frame 6 → 39.12305597113175
Frame 7 → 36.52221721899725
Frame 7 → -48.98794756468807
Frame 7 → -39.948426931994995
Frame 7 → 41.37617031018421
Frame 7 → 44.78

```
ffmpeg -r 10 -i open-closed-%03d.png -y -vf format=yuv420p morph.mp4
```